# BOExplorer
### Diary entry processing

The main information to be extracted for a diary entry is:
- The references to other entries
- Labels and least used keywords
- The economic impact
- Whether it belongs to a known category
    1. 'Licitación'
    2. 'Formalización de contrato'

#### 1. Imports

In [1]:
import json
import inspect
import functools
import collections

from lxml import etree as et

from helpers import boe
from helpers import helpers
from helpers import boe_diary_entry_processing as diary_entry
from helpers.helpers import pipe, partialmap # For code clarity

#### 2. Sample data

In [2]:
sample_data_file = 'entries_30_01_2021.json'

#### 2. Helper functions

In [3]:
boe_sections = {
    '1': 'disposiciones generales',
    '2': 'autoridades y personal',
    '2a': 'nombramientos situaciones e incidencias',
    '2b': 'oposiciones y concursos',
    '3': 'otras secciones',
    '4': 'administración de justicia',
    '5': 'anuncios',
    '5a': 'licitaciones públicas y adjudicaciones',
    '5b': 'otros anuncios particulares',
    '5c': 'anuncios particulares',
    't': 'tribunal constitucional'
}

In [4]:
def get_stats(entries, key):
    c = helpers.pipe(
        entries,
        partialmap(lambda x: x[key]),
        collections.Counter
    )
    return c

def print_overview(entries):
    print('\n', '-'*50, '\n Count by section', '\n', '-'*50)
    helpers.pipe(
        get_stats(entries, 'section'),
        lambda x: x.items(),
        partialmap(lambda x: (boe_sections[x[0].lower()], x[1])),
        partialmap(lambda x: f' {x[0]}: {x[1]}'),
        '\n'.join,
        print
    )

    print('\n', '-'*50, '\n Count by department', '\n', '-'*50)
    helpers.pipe(
        entries,
        functools.partial(filter, lambda x: x['section'] not in ('4',)),
        functools.partial(get_stats, key='department'),
        lambda x: x.items(),
        partialmap(lambda x: f' {x[0]}: {x[1]}'),
        '\n'.join,
        print
    )

#### 3. Imported processing functions

! All processing functions in the `helpers/boe_diary_entry_processing.py` module start with 'get'.

In [5]:
helpers.pipe(
    dir(diary_entry),
    functools.partial(filter, lambda x: x.startswith('get_')),
    partialmap(lambda x: diary_entry.__dict__[x]),
    partialmap(inspect.getsource),
    '\n'.join,
    print
)

def get_labels_from_tree(tree):
    tree_search = helpers.use_tree_for_search(tree)
    topics = tree_search(boe.EntryXpath.topics)
    alerts = tree_search(boe.EntryXpath.alerts)
    
    labels = helpers.pipe(topics + alerts,
        functools.partial(map, lambda x: x.xpath('text()')),
        functools.partial(map, ''.join),
        set,
        tuple)
    return labels

def get_reference_details(node):
    tree_search = helpers.use_tree_for_search(node)
    reference_type = tree_search(boe.EntryXpath.reference_type)[0]
    reference_text = tree_search(boe.EntryXpath.reference_text)[0]

    details = {
        'referenced': node.get(boe.EntryAttribute.reference_entry_id),
        'type': reference_type.text,
        'type_code': reference_type.get(boe.EntryAttribute.reference_type_code),
        'text': reference_text.text
    }
    return details

def get_references_from_tree(tree):
    tree_search = helpers.use_tree_for_search(tree)
    prev = tree_search(boe.EntryXpath.previous)


#### 4. Execution

In [6]:
with open(sample_data_file, 'r') as f:
    entries = json.load(f)

print(f' Entry count : {len(entries)}')
print_overview(entries)

 Entry count : 441

 -------------------------------------------------- 
 Count by section 
 --------------------------------------------------
 disposiciones generales: 12
 nombramientos situaciones e incidencias: 5
 oposiciones y concursos: 3
 otras secciones: 14
 administración de justicia: 252
 licitaciones públicas y adjudicaciones: 11
 otros anuncios particulares: 134
 anuncios particulares: 10

 -------------------------------------------------- 
 Count by department 
 --------------------------------------------------
 MINISTERIO DE HACIENDA: 105
 MINISTERIO DEL INTERIOR: 3
 MINISTERIO DE LA PRESIDENCIA, RELACIONES CON LAS CORTES Y MEMORIA DEMOCRÁTICA: 2
 MINISTERIO PARA LA TRANSICIÓN ECOLÓGICA Y EL RETO DEMOGRÁFICO: 3
 MINISTERIO DE ASUNTOS ECONÓMICOS Y TRANSFORMACIÓN DIGITAL: 5
 BANCO DE ESPAÑA: 2
 COMUNITAT VALENCIANA: 1
 COMUNIDAD FORAL DE NAVARRA: 3
 CONSEJO GENERAL DEL PODER JUDICIAL: 2
 MINISTERIO DE JUSTICIA: 3
 JUNTA ELECTORAL CENTRAL: 1
 ADMINISTRACIÓN LOCAL: 14
 UNIV

**4.1 Diary entry content fetch and processing of:** id, epigraph, section id, epigraph, department, pdf url, xml url, htm url, labels and references

In [7]:
diary_entry_trees = helpers.pipe(
    entries,
    partialmap(lambda x: x['xml_url']),
    partialmap(lambda x: boe.file_url_for_resource(x)),
    partialmap(lambda x: helpers.fetch_page(x)),
    partialmap(helpers.tree_from_response),
    tuple
)

In [8]:
diary_entry_labels = tuple(map(diary_entry.get_labels_from_tree, diary_entry_trees))
diary_entry_references = tuple(map(diary_entry.get_references_from_tree, diary_entry_trees))

In [9]:
print(json.dumps(entries[8], indent=0))
print('Labels: ', diary_entry_labels[8])
print('References: ', diary_entry_references[8])

{
"id": "BOE-A-2021-1353",
"epigraph": "Organizaci\u00f3n",
"section": "1",
"department": "COMUNITAT VALENCIANA",
"title": "Ley 2/2020, de 2 de diciembre, de la Informaci\u00f3n Geogr\u00e1fica y del Institut Cartogr\u00e0fic Valenci\u00e0.",
"pdf_url": "/boe/dias/2021/01/30/pdfs/BOE-A-2021-1353.pdf",
"xml_url": "/diario_boe/xml.php?id=BOE-A-2021-1353",
"htm_url": "/diario_boe/txt.php?id=BOE-A-2021-1353"
}
Labels:  ('Comunidad Valenciana', 'Geografía', 'Cartografía', 'Organización de las Comunidades Autónomas')
References:  [{'referenced': 'BOE-A-1998-243', 'type': 'DEROGA', 'type_code': '210', 'text': 'la Ley 9/1997, de 9 de diciembre', 'category': 'previous'}, {'referenced': 'BOE-A-1982-17235', 'type': 'DE CONFORMIDAD con', 'type_code': '440', 'text': 'el Estatuto aprobado por Ley Orgánica 5/1982, de 1 de julio', 'category': 'previous'}]


**4.2 Content-wise analysis to search for:** known categories and economic impact

In [10]:
title_starts = (
    'Anuncio de licitación',
    'Anuncio de formalización de contratos',
    'Anuncio de corrección de errores de la licitación',
    'Real Decreto',
)

In [11]:
titles = helpers.pipe(
    entries,
    partialmap(lambda x: x['title']),
    tuple
)
#titles

In [12]:
interesting_idxs = helpers.pipe(
    titles,
    partialmap(lambda title: map(lambda start: title.startswith(start), title_starts)),
    partialmap(any),
    enumerate,
    functools.partial(filter, lambda x: x[1]),
    partialmap(lambda x: x[0]),
    tuple
)

helpers.pipe(
    interesting_idxs,
    partialmap(lambda idx: f"[{idx}] {entries[idx]['title']}\n"),
    '\n'.join,
    print
)

[5] Real Decreto 1/2021, de 12 de enero, por el que se modifican el Plan General de Contabilidad aprobado por el Real Decreto 1514/2007, de 16 de noviembre; el Plan General de Contabilidad de Pequeñas y Medianas Empresas aprobado por el Real Decreto 1515/2007, de 16 de noviembre; las Normas para la Formulación de Cuentas Anuales Consolidadas aprobadas por el Real Decreto 1159/2010, de 17 de septiembre; y las normas de adaptación del Plan General de Contabilidad a las entidades sin fines lucrativos aprobadas por el Real Decreto 1491/2011, de 24 de octubre.

[6] Real Decreto 2/2021, de 12 de enero, por el que se aprueba el Reglamento de desarrollo de la Ley 22/2015, de 20 de julio, de Auditoría de Cuentas.

[286] Anuncio de licitación de: Mesa del Senado. Objeto: Procedimiento abierto para la adjudicación del contrato de arrendamiento y mantenimiento de impresoras y equipos multifunción del Senado. Expediente: 3/2021.

[287] Anuncio de formalización de contratos de: Delegación Especial d

In [42]:
helpers.pipe(
    diary_entry_trees,
    partialmap(helpers.use_tree_for_search),
    partialmap(lambda f: f(boe.EntryXpath.modality)),
    partialmap(lambda x: x[0].get('codigo') if len(x) == 1 else ''),
    partialmap(lambda x: EntryModality[x] if x != '' else ''),
    collections.Counter
)

Counter({'': 430,
         <EntryModality.L: 'Licitación'>: 4,
         <EntryModality.F: 'Formalización de contrato'>: 6,
         <EntryModality.O: 'Otros'>: 1})

In [52]:
helpers.pipe(
    interesting_idxs,
    partialmap(lambda idx: diary_entry_trees[idx]),
    partialmap(helpers.use_tree_for_search),
    partialmap(lambda f: f(boe.EntryXpath.modality)),
    partialmap(lambda x: x[0].get('codigo') if len(x) == 1 else ''),
    partialmap(lambda x: EntryModality[x].__str__() if x != '' else 'None'),
    collections.Counter
)

Counter({'None': 2,
         'EntryModality.L': 4,
         'EntryModality.F': 6,
         'EntryModality.O': 1})

Al parecer, todos los anuncios de licitación y formalización de contratos tienen el mismo comienzo del título. Todos ellos tienen un elemento de _procedimiento_. Hay que comprobar si otros elementos también tienen un elemento de _procedimiento_.


Tipos de documento donde aparece una cuantía de dinero, y partes del documento donde se recogía la cuantía:

- Formalización de contrato (Ministerio de asuntos económicos y transformación digital
  ```html
  <dl>
    <dt>13.1.1) Valor de la oferta seleccionada: </dt>
    <dd>1.746.360,00 euros.</dd>
  </dl>
  ```
- Anuncio de licitación
  ```html
  <dt>8. Valor estimado:</dt>
    <dd>580.800,00 euros.</dd>
  <dt>9. Informaci&#243;n sobre las variantes:</dt>
  ```
  Había otros lugares donde aparecían cantidades monetarias mencionadas
  ```html
  <dd>
      Cifra anual de negocio (el volumen anual de negocios de cualquiera de los tres &#250;ltimos a&#241;os 
      concluidos deber&#225; ser, al menos, una vez y media el valor anual medio del contrato
      (180.000,00 euros, IVA no incluido)).
  </dd>
  <dt>11.5) Situaci&#243;n t&#233;cnica y profesional:</dt>
  <dd>
      Trabajos realizados (la facturaci&#243;n anual acumulada por la realizaci&#243;n de estos trabajos en 
      cualquiera de los tres &#250;ltimos a&#241;os concluidos deber&#225; ser, al menos, igual al 70% del
      valor anual medio del contrato (84.000,00 euros, IVA no incluido)).
  </dd>
  ```
- Anuncio de formalización de contrato (Ministerio de Hacienda)
  ```html
  <dl>
      <dt>13.1) Valor de la oferta seleccionada: </dt>
      <dd>19.052,76 euros.</dd>
      <dt>13.2) Valor de la oferta de mayor coste: </dt>
      <dd>24.047,00 euros.</dd>
      <dt>13.3) Valor de la oferta de menor coste: </dt>
      <dd>19.052,76 euros.</dd>
  </dl>
  ```
- Anuncio de licitación (ADIF)
  ```html
  <dt>8. Valor estimado:</dt>
  <dd>4.564.597,84 euros.</dd>
  ```
- Anuncio de formalización de contratos
  ```html
  <dl>
    <dt>13.1) Valor de la oferta seleccionada: </dt>
    <dd>346.679,00 euros.</dd>
  </dl>
  ```
- Anuncio de la formalización de contrato (obra de emergencia)
  ```html
  <dd>
    <dl>
      <dt>13.1) Valor de la oferta seleccionada: </dt>
      <dd>12.956,51 euros.</dd>
    </dl>
  </dd>
  ```
- Anuncio de formalización de contratos (Instituto Nacional de Estadística)
  ```html
  <dd>
    <dl>
      <dt>13.1) Valor de la oferta seleccionada: </dt>
      <dd>81.520,67 euros.</dd>
      <dt>13.2) Valor de la oferta de mayor coste: </dt>
      <dd>110.677,68 euros.</dd>
      <dt>13.3) Valor de la oferta de menor coste: </dt>
      <dd>0,00 euros.</dd>
    </dl>
  </dd>
  ```
- Anuncio de formalización de contratos (Instituto Nacional de Estadística)
  ```html
  <dl>
      <dt>13.1.1) Valor de la oferta seleccionada: </dt>
      <dd>1.746.360,00 euros.</dd>
  </dl>
  ```
- Anuncio de formalización de contratos (Dirección General  de la Entidad Pública Empresarial RED.ES
  ```html
  <dl>
    <dt>13.1) Valor de la oferta seleccionada: </dt>
    <dd>373.951,34 euros.</dd>
  </dl>
  ```
- Anuncio de licitaciòn de: Presidencia de la Agencia Estatal Consejo Superior de Investigaciones Científicas
  Sin cuantía económica asociada, tenía el procedimiento abierto:
  ```html
  <procedimiento codigo="1">Abierto</procedimiento>
  ```
- Anuncio de licitación de: Subdirección de Compras de la Sociedad Estatal Correos y Telégrafos
  Con cuantía económica asociada, y
  ```html
  <procedimiento codigo="4">Negociado con publicidad</procedimiento>
  ```
  ```html
  <dt>8. Valor estimado:</dt>
  <dd>1.103.928,85 euros.</dd>
  ```

In [76]:
helpers.pipe(
    (interesting_idxs[12],),
    partialmap(lambda idx: diary_entry_trees[idx]),
    # partialmap(helpers.use_tree_for_search),
    # partialmap(lambda f: f('/documento/analisis')),
    partialmap(lambda x: et.tostring(x).decode('utf8')),
    '\n\n'.join,
    #print
)
None

In [16]:
# main title begginings
helpers.pipe(
    entries,
    partialmap(lambda x: x['title'][:min(len('Resolución'), len(x['title']))]),
    collections.Counter
)

Counter({'Resolución': 27,
         'Orden INT/': 1,
         'Orden PCM/': 1,
         'Real Decre': 2,
         'Circular 1': 1,
         'Ley 2/2020': 1,
         'Ley Foral ': 3,
         'Acuerdo de': 2,
         'Orden JUS/': 3,
         'Orden CSM/': 1,
         'Corrección': 1,
         'Acuerdo 5/': 1,
         'ALCALA DE ': 1,
         'ALCAÑIZ': 1,
         'ALGECIRAS': 1,
         'ALICANTE/A': 1,
         'ALZIRA': 1,
         'AMPOSTA': 1,
         'ARENYS MAR': 2,
         'ARGANDA DE': 1,
         'AVILÉS': 3,
         'AZPEITIA': 1,
         'BADAJOZ': 3,
         'BADALONA': 3,
         'BAENA': 1,
         'BARAKALDO': 2,
         'BARCELONA': 17,
         'BARCO DE V': 1,
         'BENAVENTE': 1,
         'BILBAO': 7,
         'BOLTAÑA': 1,
         'BURGOS': 1,
         'CALAHORRA': 1,
         'CARLET': 2,
         'CARMONA': 1,
         'CASTELLON ': 1,
         'CASTUERA': 1,
         'CATARROJA': 1,
         'CERDANYOLA': 3,
         'CERVERA': 1,
         'COL